In [2]:
import testsets
import evaluation
import twokenize
import sklearn.feature_extraction
import sklearn
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import textPreprocessor01
import nltk
from nltk.stem import *
from nltk.probability import FreqDist
from nltk.corpus import sentiwordnet as swn

from gensim.models import word2vec
import word2vecReader
from sklearn.naive_bayes import GaussianNB

C:\Users\yanch\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
# TODO: load training data
def read_training_data(training_data):
  id_gts = {}
  with open(training_data, 'r', encoding='utf-8') as f:
    for line in f:
      fields = line.split('\t')
      tweetid = fields[0]
      gt = fields[1]
      content = fields[2].strip()
      id_gts[tweetid] = gt, content

  return id_gts
traindic = read_training_data('twitter-training-data1.txt')

In [4]:
def perprocessing(tdic):
    new_dic = {}
    for line in tdic:
        id = line
        gt = tdic[line][0]
        raw = ' '.join(twokenize.tokenizeRawTweetText(tdic[line][1]))
        text = twokenize.normalizeTextForTagger(raw)
        text_tk = twokenize.tokenize(text)
        # print(text_tk)
        newtext = ' '.join(text_tk)
        newtext = textPreprocessor01.replaceall(newtext)
        new_dic[id] = gt, newtext
        # print(type(tdic[line][1]))
        # print(line)
        # print(type(line))
        # print(type(newtext))
        # print(newtext)
    return new_dic

In [5]:
def get_train_corpus(new_dic):
    traincorpus = []
    for line in new_dic:
        # print('after perprocessing')
        # print(line)
        # print(new_dic[line])
        # print(new_dic[line][1])
        traincorpus.append(new_dic[line][1])
    return traincorpus

In [6]:
def get_split_corpus(new_dic):
    split_traincorpus = []
    for line in new_dic:
        split_traincorpus.append(new_dic[line][1].split())
    return split_traincorpus

In [7]:
# TODO extract features
def get_ngrams(corpus):
    vectorizer = CountVectorizer(stop_words='english')
    # vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 2))
    X = vectorizer.fit_transform(corpus)
    # print(vectorizer.vocabulary_)
    # print(vectorizer.vocabulary_.keys())
    # print('ngram----')
    # print(X.todense())
    # print(len(X.todense()))
    # X.todense()
    # print(X.toarray())
    return X.todense()

In [8]:
def get_tfidf(corpus):
    vectorizer = CountVectorizer(stop_words='english')
    # vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 2))
    X = vectorizer.fit_transform(corpus)
    # print(vectorizer.vocabulary_)
    # print(vectorizer.vocabulary_.keys())
    tfidf = TfidfVectorizer(vocabulary=list(vectorizer.vocabulary_.keys()), stop_words='english')
    # tfidf = TfidfVectorizer(vocabulary=list(vectorizer.vocabulary_.keys()), stop_words='english', ngram_range=(1, 2))
    tfs = tfidf.fit_transform(corpus)
    # print('tfidf------------------------------------')
    # print(tfs.todense())
    # print(len(tfs.todense()))
    return tfs.todense()

In [9]:
def word_embedding2(split_corpus):
    print('word embedding2 --------------------')
    all = []
    for i in split_corpus:
        model = word2vec.Word2Vec([i], min_count=1)
        # print(model.vocabulary)
        # print(model.wv.vocab)

        s = model.wv.syn0

        # print(s)
        # print(len(i))   # i is each tweet.  len here is 20,25,22,14
        # print(len(s))  # len here is 22 30 34 28. why the length here is not equal to original tweet.
        ans = list(map(sum, zip(*s)))  # sum of them
        # print(ans)
        # print(len(ans))
        all.append(ans)
    # print(len(all))  # 4
    # print(all)
    return all

In [10]:
def senti_bi_lexicon(split_corpus):
    def inputfile(file):
        with open(file, 'r') as my_file:
            words = [every_line.rstrip() for every_line in my_file]
            return words

    def count_p_n(mylist):
        pos_num = 0
        neg_num = 0
        positive = inputfile('positive-words.txt')
        negative = inputfile('negative-words.txt')
        p_dic = FreqDist(positive)
        n_dic = FreqDist(negative)
        for word in mylist:
            pos_num += p_dic[word]
            neg_num += n_dic[word]
        return pos_num, neg_num

    P_N = []
    for line in split_corpus:
        p_num_all = n_num_all = 0
        p_n_num = count_p_n(line)
        p_num_all += p_n_num[0]
        n_num_all += p_n_num[1]
        # p_n_dif = p_n_num[0] - p_n_num[1]
        # if p_n_dif > 0:
        #     P_N.append(1)  # positive
        # elif p_n_dif < 0:
        #     P_N.append(-1)  # negative
        # else:
        #     P_N.append(0)  # neutral
        P_N.append([p_num_all, n_num_all])
    # print('senti_binary_lex--------------------')
    # print(P_N)
    # print(len(P_N))
    return P_N

In [11]:
newdic = perprocessing(traindic)
train_corpus = get_train_corpus(newdic)
split_corpus = get_split_corpus(newdic)

In [12]:
for classifier in ['myclassifier1', 'myclassifier2', 'myclassifier3']: # You may rename the names of the classifiers to something more descriptive
    if classifier == 'myclassifier1':
        print('Training ' + classifier)
        # TODO: extract features for training classifier1
        # TODO: train sentiment classifier1
        f1 = get_ngrams(train_corpus)
        F1 = np.array(f1)
        f2 = get_tfidf(train_corpus)
        F2 = np.array(f2)

        f3 = senti_bi_lexicon(split_corpus)
        F3 = np.array(f3)
        # # print(f3)
        # # print(np.array(f3))
        # # wordembedding(split_corpus)
        # f4 = word_embedding2(split_corpus)
        # # F4 = np.array(f4)
        #
        # t1_f3 = senti_bi_lexicon(t1s_corpus)
        # t1_F3 = np.array(t1_f3)
        #
        X = F3
        #
        labels_to_array = {"positive": 1, "negative": -1, "neutral": 0}
        labels = [labels_to_array[newdic[tweet][0]] for tweet in newdic]
        # print(labels)
        Y = np.array(labels)
        #
        model = GaussianNB()
        model.fit(X, Y)
        #
        # # print(clf.predict([[]]))
        #
        # ans_num = clf.predict(t1_F3)
        # # print(ans)
        # # print(len(ans))
        # array_to_labels = {1:"positive" ,  -1 :"negative",  0:"neutral"}
        # labels = [array_to_labels[i] for i in ans_num]
        # # print(labels)
        # # ans_dic = {}
        # ans_dictionary = dict(zip(list(test1dic.keys()), labels))
        # print(ans_dictionary)

        # num_right = 0
        # for count,i in enumerate(newdic):
        #     # print(i)
        #     # print(count)
        #     # print(newdic[i][0])
        #     # print(type(sentlist[count]))
        #     if sentlist[count] == 1 and newdic[i][0] == 'positive':
        #         # print('right ans')
        #         num_right += 1
        #     if sentlist[count] == 0 and newdic[i][0] == 'neutral':
        #         num_right += 1
        #     if sentlist[count] == -1 and newdic[i][0] == 'negative':
        #         num_right += 1
        # print(num_right/len(sentlist))



    elif classifier == 'myclassifier2':
        print('Training ' + classifier)
        # TODO: extract features for training classifier2
        # TODO: train sentiment classifier2
    elif classifier == 'myclassifier3':
        print('Training ' + classifier)
        # TODO: extract features for training classifier3
        # TODO: train sentiment classifier3

    for testset in testsets.testsets:
        # TODO: classify tweets in test set
        test = read_training_data(testset)

        testdic = perprocessing(test)
        t_corpus = get_train_corpus(testdic)
        ts_corpus = get_split_corpus(testdic)

        t_f3 = senti_bi_lexicon(ts_corpus)
        t_F3 = np.array(t_f3)

        ans_num = model.predict(t_F3)
        # # print(ans)
        # # print(len(ans))
        array_to_labels = {1:"positive" ,  -1 :"negative",  0:"neutral"}
        labels = [array_to_labels[i] for i in ans_num]
        # # print(labels)
        # # ans_dic = {}
        predictions = dict(zip(list(testdic.keys()), labels))
        # print(ans_dictionary)


        # predictions = {'163361196206957578': 'neutral', '768006053969268950': 'neutral', '742616104384772304': 'neutral', '102313285628711403': 'neutral', '653274888624828198': 'neutral'} # TODO: Remove this line, 'predictions' should be populated with the outputs of your classifier
        # predictions = ans_dictionary
        evaluation.evaluate(predictions, testset, classifier)

        evaluation.confusion(predictions, testset, classifier)

Training myclassifier1
twitter-test1.txt (myclassifier1): 0.538
            positive  negative  neutral
positive    0.698     0.077     0.225     
negative    0.227     0.489     0.284     
neutral     0.292     0.121     0.587     

twitter-test2.txt (myclassifier1): 0.499
            positive  negative  neutral
positive    0.745     0.052     0.203     
negative    0.339     0.399     0.261     
neutral     0.436     0.082     0.483     

twitter-test3.txt (myclassifier1): 0.516
            positive  negative  neutral
positive    0.676     0.091     0.233     
negative    0.211     0.425     0.364     
neutral     0.351     0.100     0.549     

Training myclassifier2
twitter-test1.txt (myclassifier2): 0.538
            positive  negative  neutral
positive    0.698     0.077     0.225     
negative    0.227     0.489     0.284     
neutral     0.292     0.121     0.587     

twitter-test2.txt (myclassifier2): 0.499
            positive  negative  neutral
positive    0.745     0.052  